In [98]:
import pandas as pd
import re
applications = pd.read_csv("submissions.csv")
applications["Name"]
profiles = pd.DataFrame(columns = ["name", "contact", "bio", "subjects", "days", "form_name"])


In [99]:
applications = applications[applications["Unnamed: 11"].isin(["done"])==False]
profiles["form_name"] = applications["Name"]+" Submision Form"
profiles["name"] = applications["Name"]
profiles = profiles.set_index("name")
applications = applications.set_index("Name")

In [100]:
def get_grade(string):
    for i in ["11", "junior"]:
        if re.search(i, string, re.IGNORECASE):
            return "Junior"
    for i in ["12", "Senior"]:
        if re.search(i, string, re.IGNORECASE):
            return "Senior"
    for i in ["10", "sophomore"]:
        if re.search(i, string, re.IGNORECASE):
            return "Sophomore"
    for i in ["9", "Freshman"]:
        if re.search(i, string, re.IGNORECASE):
            return "Freshman"
    return "ERROR"

In [101]:
for index, row in applications.iterrows():
    print(index)
'''
profiles["contact"] = "Grade:" + (get_grade(profile)) Junior

Highest Education: Stevenson High School

Email: shanky.bala26@gmail.com

Phone: (224) 532-3619

Tutoring Experience: 3 years
'''

Shashank Bala
Jihwan Kim
Sri Medakkar
Amber Dorn
Sydney Manshum
Sandra Suresh
Anushka Desai
Sandra Suresh
Madeline Kegel
James Shin
Kristine Chi
Jasmine Zhang
Kaylee Brady
Andy Cha
Jessica Zhang
Renee Shiao
Kyle Chiu


'\nprofiles["contact"] = "Grade:" + (get_grade(profile)) Junior\n\nHighest Education: Stevenson High School\n\nEmail:\xa0shanky.bala26@gmail.com\n\nPhone:\xa0(224) 532-3619\n\nTutoring Experience: 3\xa0years\n'

In [102]:
applications

,Submitted On,Grade,School,Email Address,Phone,Position,Subjects,Availability,Experience,Biography,Unnamed: 11
Name,,,,,,,,,,,
Shashank Bala,01/25/2018 20:42:26,11,Adlai E. Stevenson High School,shanky.bala26@gmail.com,(224) 532-3619,Tutor,"Elementary School Level Subjects, Junior High ...","Sunday, Saturday",Shashank has been teaching chess for elementar...,Shashank Bala is a Junior at Stevenson High Sc...,NaN
Jihwan Kim,02/06/2018 21:21:28,Sophomore,Stevenson high school,Jihwan123jhk@gmail.com,(847) 429-8610,Tutor,"Elementary School Level Subjects, Junior High ...","Friday, Saturday",I have almost no tutoring experience except he...,Jihwan Kim is a sophomore at Stevenson high sc...,NaN
Sri Medakkar,02/12/2018 0:09:40,11,Adlai E. Stevenson High School,medakkar3@gmail.com,(732) 513-9344,Both,"Elementary School Level Subjects, Junior High ...","Sunday, Saturday",I have helped friends at school in subjects li...,Sri Medakkar is a junior at Adlai E. Stevenson...,NaN
Amber Dorn,02/16/2018 17:21:02,11,Carmel Catholic High School,amberdorn29@yahoo.com,(224) 247-2651,Both,"Elementary School Level Subjects, Junior High ...","Tuesday, Friday",I have been tutoring kids in my neighborhood i...,Amber Dorn is a Junior at Carmel Catholic High...,NaN
Sydney Manshum,02/16/2018 17:28:12,11,Carmel Catholic High School,Syd_manshum@yahoo.com,(847) 542-7936,Select,"Elementary School Level Subjects, Junior High ...","Sunday, Monday, Tuesday, Wednesday, Thursday, ...",I tutor my cousin who goes to St. Gilbert grad...,Sydney Manshum is a junior at Carmel High Scho...,NaN
Sandra Suresh,02/16/2018 18:22:28,11,Adlai E. Stevenson High School,ssuresh9@students.d125.org,(224) 392-2197,Tutor,"Elementary School Level Subjects, Science, Mat...","Monday, Tuesday, Friday, Saturday",I occasionally tutor my brother and his friend...,Sandra Suresh is a Junior at Stevenson High sc...,NaN
Anushka Desai,02/16/2018 21:21:22,Junior / 11th,Mundelein High School,anushkadesai@yahoo.com,(847) 521-8350,Both,"Elementary School Level Subjects, Junior High ...",Sunday,I have tutoring for 2 years now through the pe...,Anushka Desai is a junior at Mundelein High Sc...,NaN
Sandra Suresh,02/22/2018 8:57:45,11,Adali E. Stevenson,ssuresh9@students.d125.org,(224) 392-2197,Both,"Elementary School Level Subjects, Junior High ...","Monday, Tuesday, Wednesday, Friday, Saturday",I have some experience tutoring my brother and...,Sandra Suresh is a junior at Adlai E. Stevenso...,NaN
Madeline Kegel,02/25/2018 19:53:07,11,Carmel Catholic High School,mkegel.2019@carmelhs.org,(847) 970-1550,Tutor,"Elementary School Level Subjects, Junior High ...","Sunday, Monday, Saturday",I started tutoring for my school this year in ...,Maddy Kegel is a junior at Carmel Catholic Hig...,NaN
